In [ ]:
#@title 一、下载源码
# # 并解压

In [ ]:
#@title 二、安装依赖
# 因为google升级了pytorch 但是代码是用的torch2.0.1写得，所以需要降低版本。
#torch版本如果不是2.0.1+cu118，，则重新安装该版本
pip install -r requirements.txt

In [ ]:
#@title 三、数据处理
# 1、 用split_video.py 切割视频
# 2、 用sample.py 生成 filelists

# !wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "./face_detection/detection/sfd/s3fd.pth"
# 3、数据处理
python  preprocess.py  --data_root  ./data  --preprocessed_root ./preprocessed_data

In [ ]:
#@title 四、判别器训练
# 如果爆显存 ，请将 hparams.py的 batch_size 改成4 或者更小。
python color_syncnet_train.py --data_root  preprocessed_data  --checkpoint_dir color_checkpoints


In [ ]:
#@title 五、生成器训练
# 如果爆显存 ，请将 hparams.py的 batch_size 改成4 或者更小。
python wav2lip_train.py  --data_root  preprocessed_data  --checkpoint_dir  final_checkpionts  --syncnet_checkpoint_path  color_checkpoints/checkpoint_step000699000.pth

In [ ]:
#@title 六、推理
# 如果爆显存 ，请将inference.py的 batch_size 改成4 或者更小。
python inference.py --checkpoint_path  final_checkpionts/checkpoint_step000360000.pth  --face ./input/555.mp4  --audio  ./input/555.wav

In [ ]:
# 查看文件夹下面， 每个子文件夹下面的文件数量
for d in */ ; do echo "$d"; find "$d" -maxdepth 1 -type f | wc -l; done
#  查看音频采样
ffmpeg -i audio.wav

In [ ]:
# 跨区域拷贝数据
# 拷贝到那个区的项目
ssh -p 39861 root@connect.westb.seetacloud.com
# 从此项目拷贝到目标项目中
scp -rP 39861 /root/autodl-tmp/wav2lip384/final_checkpionts/checkpoint_step000480000.pth   root@connect.westb.seetacloud.com:/rootautodl-tmp/wav2lip384

In [ ]:
# tensorboard显示loss
tensorboard --port 7860 --logdir /root/autodl-tmp/

In [ ]:
# 转成16000赫兹